In [70]:
import pandas as pd
import sqlite3
import sqlalchemy

In [80]:
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
interactions = pd.read_sql_query("SELECT * FROM interactions", sqlite)
interactions['createdAt'] = pd.to_datetime(interactions['createdAt'])
interactions = interactions[(interactions['createdAt'] > '2021-07-10')]

#create list of hashes of sessions that start too smal and filter all following events
startTooSmallHashes = interactions[(interactions['event'].str.startswith('notLargeEnough'))]['hash'].values.tolist()
interactions = interactions[~interactions['hash'].isin(startTooSmallHashes)]

surveys = pd.read_sql_query("SELECT * FROM Surveys", sqlite)
surveys['createdAt'] = pd.to_datetime(surveys['createdAt'])
surveys = surveys[(surveys['createdAt'] > '2021-07-10')]

In [88]:
 interactions[interactions['path'] == "separated"]

,id,hash,path,event,element,createdAt,updatedAt,diff
3449,3450,1Utv-BSN0wvWSdzqHaE63,separated,sessionStart,None,2021-07-14 14:21:49.276000+00:00,2021-07-14 14:21:49.276 +00:00,0 days 00:00:00.886000
3450,3451,1Utv-BSN0wvWSdzqHaE63,separated,scrollPosition,0,2021-07-14 14:21:52.332000+00:00,2021-07-14 14:21:52.332 +00:00,0 days 00:00:03.056000
3451,3452,1Utv-BSN0wvWSdzqHaE63,separated,scrollPosition,0,2021-07-14 14:21:55.333000+00:00,2021-07-14 14:21:55.333 +00:00,0 days 00:00:03.001000
3452,3453,1Utv-BSN0wvWSdzqHaE63,separated,scrollPosition,0,2021-07-14 14:21:58.335000+00:00,2021-07-14 14:21:58.335 +00:00,0 days 00:00:03.002000
3453,3454,1Utv-BSN0wvWSdzqHaE63,separated,scrollPosition,0,2021-07-14 14:22:01.340000+00:00,2021-07-14 14:22:01.340 +00:00,0 days 00:00:03.005000
...,...,...,...,...,...,...,...,...
98814,98815,WoMOdLLFNSHDkA3dy9kOO,separated,scrollPosition,654,2022-01-03 14:24:29.035000+00:00,2022-01-03 14:24:29.035 +00:00,0 days 00:01:01.205000
98815,98816,WoMOdLLFNSHDkA3dy9kOO,separated,scrollPosition,654,2022-01-03 14:25:23.849000+00:00,2022-01-03 14:25:23.849 +00:00,0 days 00:00:54.814000
98816,98817,WoMOdLLFNSHDkA3dy9kOO,separated,scrollPosition,654,2022-01-03 14:26:15.125000+00:00,2022-01-03 14:26:15.125 +00:00,0 days 00:00:51.276000
98817,98818,WoMOdLLFNSHDkA3dy9kOO,separated,scrollPosition,654,2022-01-03 14:27:07.374000+00:00,2022-01-03 14:27:07.374 +00:00,0 days 00:00:52.249000


In [81]:
#with what view did people start?
interactions.loc[interactions['event'] == 'sessionStart'].groupby(['path']).count()

,id,hash,event,element,createdAt,updatedAt
path,,,,,,
contextual,329,329,329,0,329,329
embedded,181,181,181,0,181,181
separated,161,161,161,0,161,161


In [85]:
#how many memories were submitted
interactions.loc[interactions['event'] == 'sendMemory'].groupby(['path']).count()

,id,hash,event,element,createdAt,updatedAt,diff
path,,,,,,,
embedded,12,12,12,0,12,12,12
separated,6,6,6,0,6,6,6


In [74]:
#how many surveys were submitted
surveys.groupby(['survey']).count()

,id,hash,checkboxes,comment,createdAt,updatedAt
survey,,,,,,
reading,11,11,11,0,11,11
submitting,17,17,17,17,17,17


In [82]:
#how many memories/policies did people click?
interactions.loc[interactions['event'].str.startswith('open')].groupby(['path']).count()

,id,hash,event,element,createdAt,updatedAt
path,,,,,,
contextual,965,965,965,965,965,965
embedded,739,739,739,739,739,739


In [84]:
#how much time did people spend in each view (independent of session start view)
interactions['diff'] = interactions['createdAt'].diff()
interactions = interactions.dropna(subset=['diff'])
grouped = interactions.loc[interactions["event"] != "sessionStart"].groupby(['path'])
grouped['diff'].max()

path
contextual   0 days 08:33:11.572000
embedded     0 days 02:33:25.174000
separated    0 days 15:54:11.906000
Name: diff, dtype: timedelta64[ns]

In [89]:
#from what view to what view did people migrate?
interactions.loc[interactions['event'] == 'changeView'].groupby(['path','element']).count()

id  hash  event  createdAt  updatedAt  diff
path       element                                                
contextual contextual   7     5      7          7          7     7
           embedded    62    62     62         62         62    62
           separated   46    46     46         46         46    46
embedded   contextual  60    60     60         60         60    60
           embedded     7     7      7          7          7     7
           separated   30    30     30         30         30    30
separated  contextual  32    32     32         32         32    32
           embedded    49    49     49         49         49    49
           separated    1     1      1          1          1     1

In [106]:
#how many people do not use a screen alrge enough
interactions.loc[interactions['event'].str.startswith('n')]

,id,hash,path,event,element,createdAt,updatedAt
3156,3157,MbLLCyVJr2KkpsB2zm2u9,embedded,notLargeEnough,None,2021-07-14 14:04:57.141000+00:00,2021-07-14 14:04:57.141 +00:00
3159,3160,3YKALopb95Z5-YNIU91fj,embedded,notLargeEnough,None,2021-07-14 14:05:01.433000+00:00,2021-07-14 14:05:01.433 +00:00
3162,3163,omVBP332fb1gyE868-s09,embedded,notLargeEnough,None,2021-07-14 14:05:04.576000+00:00,2021-07-14 14:05:04.576 +00:00
5192,5193,hg7z9m-9VedEE9BCxQ9QY,separated,notLargeEnough,None,2021-08-21 23:56:47.492000+00:00,2021-08-21 23:56:47.492 +00:00
5482,5483,0SvbdBZ3B5yS-nnC-W6a8,contextual,notLargeEnough,None,2021-10-01 09:25:58.420000+00:00,2021-10-01 09:25:58.420 +00:00
...,...,...,...,...,...,...,...
98340,98341,L38QQaFzb8q3HxM50a6pq,contextual,notLargeEnough,None,2021-12-26 21:34:43.531000+00:00,2021-12-26 21:34:43.531 +00:00
98342,98343,bgm4idZxG5_HKT6UdxfnD,separated,notLargeEnough,None,2021-12-27 08:27:17.060000+00:00,2021-12-27 08:27:17.060 +00:00
98370,98371,a0ePVvsu0AcJlC-_0Yt9L,contextual,notLargeEnough,None,2021-12-27 16:08:33.645000+00:00,2021-12-27 16:08:33.645 +00:00
98390,98391,3qTFAll9VSGWNK1fASwL3,contextual,notLargeEnough,None,2021-12-28 08:18:09.876000+00:00,2021-12-28 08:18:09.876 +00:00
